## Advent of Code

The almanac (your puzzle input) lists all of the seeds that need to be planted. It also lists what type of soil to use with each kind of seed, what type of fertilizer to use with each kind of soil, what type of water to use with each kind of fertilizer, and so on. Every type of seed, soil, fertilizer and so on is identified with a number, but numbers are reused by each category - that is, soil 123 and fertilizer 123 aren't necessarily related to each other.

For example:

In [ ]:
from fastcore.utils import L
from aocd import get_data

inp = get_data(day=5, year=2023)
samp = '''seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4'''
samp

'seeds: 79 14 55 13\n\nseed-to-soil map:\n50 98 2\n52 50 48\n\nsoil-to-fertilizer map:\n0 15 37\n37 52 2\n39 0 15\n\nfertilizer-to-water map:\n49 53 8\n0 11 42\n42 0 7\n57 7 4\n\nwater-to-light map:\n88 18 7\n18 25 70\n\nlight-to-temperature map:\n45 77 23\n81 45 19\n68 64 13\n\ntemperature-to-humidity map:\n0 69 1\n1 0 69\n\nhumidity-to-location map:\n60 56 37\n56 93 4'

How to fix this error? The module is in the parent directory.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

The issue is with the import syntax. In Python, you can't use `../` in import statements like you would in file paths.

To import from a parent directory, you have a couple of options:

1. **Use a relative import with a dot notation:**
   ```python
   from ..utils import *
   ```
   (This works if you're running this as part of a package)

2. **Add the parent directory to your Python path:**
   ```python
   import sys
   sys.path.append('..')
   from utils import *
   ```

Which approach would you like to try? The second option is usually more straightforward when working in notebooks or standalone scripts.

The almanac starts by listing which seeds need to be planted: seeds 79, 14, 55, and 13.

The rest of the almanac contains a list of maps which describe how to convert numbers from a source category into numbers in a destination category. That is, the section that starts with seed-to-soil map: describes how to convert a seed number (the source) to a soil number (the destination). This lets the gardener and his team know which soil to use with which seeds, which water to use with which fertilizer, and so on.

Rather than list every source number and its corresponding destination number one by one, the maps describe entire ranges of numbers that can be converted. Each line within a map contains three numbers: the destination range start, the source range start, and the range length.

Consider again the example seed-to-soil map:
```
50 98 2
52 50 48
```
The first line has a destination range start of 50, a source range start of 98, and a range length of 2. This line means that the source range starts at 98 and contains two values: 98 and 99. The destination range is the same length, but it starts at 50, so its two values are 50 and 51. With this information, you know that seed number 98 corresponds to soil number 50 and that seed number 99 corresponds to soil number 51.

The second line means that the source range starts at 50 and contains 48 values: 50, 51, ..., 96, 97. This corresponds to a destination range starting at 52 and also containing 48 values: 52, 53, ..., 98, 99. So, seed number 53 corresponds to soil number 55.

Any source numbers that aren't mapped correspond to the same destination number. So, seed number 10 corresponds to soil number 10.

So, the entire list of seed numbers and their corresponding soil numbers looks like this:
```
seed  soil
0     0
1     1
...   ...
48    48
49    49
50    52
51    53
...   ...
96    98
97    99
98    50
99    51
```
With this map, you can look up the soil number required for each initial seed number:

Seed number 79 corresponds to soil number 81.
Seed number 14 corresponds to soil number 14.
Seed number 55 corresponds to soil number 57.
Seed number 13 corresponds to soil number 13.
The gardener and his team want to get started as soon as possible, so they'd like to know the closest location that needs a seed. Using these maps, find the lowest location number that corresponds to any of the initial seeds. To do this, you'll need to convert each seed number through other categories until you can find its corresponding location number. In this example, the corresponding types are:

Seed 79, soil 81, fertilizer 81, water 81, light 74, temperature 78, humidity 78, location 82.
Seed 14, soil 14, fertilizer 53, water 49, light 42, temperature 42, humidity 43, location 43.
Seed 55, soil 57, fertilizer 57, water 53, light 46, temperature 82, humidity 82, location 86.
Seed 13, soil 13, fertilizer 52, water 41, light 34, temperature 34, humidity 35, location 35.
So, the lowest location number in this example is 35.

What is the lowest location number that corresponds to any of the initial seed numbers?

Idea:
- Imlement a custom dictionary which will map location numbers.

In Python, is there a dictionary-like class which would map whole ranges of numbers?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

There isn't a built-in dictionary class in Python's standard library that maps whole ranges of numbers, but you have a few good options:

1. **Build your own custom class** - You could create a class that stores range mappings (source start, destination start, length) and implements `__getitem__` to look up values. This is probably the most efficient approach for your Advent of Code problem.

2. **Use a library** - There are some third-party libraries like `intervaltree` or `portion` that handle interval/range operations, but they might be overkill for this problem.

3. **Use a regular dict with a fallback** - For smaller ranges, you could populate a regular dictionary, but with potentially large ranges (like in AoC), this could use too much memory.

For your problem, I'd recommend option 1 - a custom class that stores the mapping rules and computes the destination value on demand. This way you're not storing millions of individual key-value pairs.

Would you like to try implementing a custom class for this, or would you like me to show you a small example to get started?

Show me a small example for option 1 to get started.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Here's a small example to get you started:

```python
class RangeDict:
    def __init__(self):
        self.ranges = []
    
    def add_range(self, dest_start, source_start, length):
        self.ranges.append((dest_start, source_start, length))
    
    def __getitem__(self, key):
        for dest_start, source_start, length in self.ranges:
            if source_start <= key < source_start + length:
                offset = key - source_start
                return dest_start + offset
        return key  # If no mapping found, return the key itself
```

This gives you a basic structure. You can add ranges with `add_range()` and then look up values using square brackets like `rd[98]`.

Try implementing this and testing it with the seed-to-soil map from your sample data. Does this approach make sense for what you need?

In [ ]:
class RangeDict:
    def __init__(self):
        self.ranges = []
    
    def add_range(self, dest_start, source_start, length):
        self.ranges.append((dest_start, source_start, length))
    
    def __getitem__(self, key):
        for dest_start, source_start, length in self.ranges:
            if source_start <= key < source_start + length:
                offset = key - source_start
                return dest_start + offset
        return key  # If no mapping found, return the key itself

rd = RangeDict()
rd.add_range(50, 98, 2)
rd.add_range(52, 50, 48)
L(79, 14, 55, 13).map(rd.__getitem__)

[81, 14, 57, 13]

In [ ]:
xs = L(samp.splitlines())
xs

['seeds: 79 14 55 13', '', 'seed-to-soil map:', '50 98 2', '52 50 48', '', 'soil-to-fertilizer map:', '0 15 37', '37 52 2', '39 0 15', '', 'fertilizer-to-water map:', '49 53 8', '0 11 42', '42 0 7', '57 7 4', '', 'water-to-light map:', '88 18 7', '18 25 70', '', 'light-to-temperature map:', '45 77 23', '81 45 19', '68 64 13', '', 'temperature-to-humidity map:', '0 69 1', '1 0 69', '', 'humidity-to-location map:', '60 56 37', '56 93 4']

How can I "chunk" the "xs" list? In the example above, I'd like to create groups of lines. The groups are delimited by an empty line.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

There are several ways to chunk by empty lines. Here are a couple of approaches:

1. **Using `itertools.groupby`** - Group consecutive lines based on whether they're empty or not
2. **Manual loop** - Iterate through and build chunks when you hit empty lines
3. **Split on empty strings** - Similar to how you'd split a string

Since you're using fastcore's `L`, you might find option 2 or 3 most straightforward. 

Would you like a hint on how to implement one of these, or would you prefer to try it yourself first?

But I would like to split the list of strings where an empty string serves as a delimiter.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You can split a list on a delimiter by iterating through and collecting items between delimiters. Here's a small helper function approach:

```python
def split_on(lst, delimiter):
    result = []
    current = []
    for item in lst:
        if item == delimiter:
            if current:
                result.append(current)
                current = []
        else:
            current.append(item)
    if current:
        result.append(current)
    return result
```

Try using `split_on(xs, '')` to split your list on empty strings. Does this give you the chunks you're looking for?

In [ ]:
def split_on(lst, delimiter):
    result = []
    current = []
    for item in lst:
        if item == delimiter:
            if current:
                result.append(current)
                current = []
        else:
            current.append(item)
    if current:
        result.append(current)
    return result
ys = split_on(xs, '')
ys

[['seeds: 79 14 55 13'],
 ['seed-to-soil map:', '50 98 2', '52 50 48'],
 ['soil-to-fertilizer map:', '0 15 37', '37 52 2', '39 0 15'],
 ['fertilizer-to-water map:', '49 53 8', '0 11 42', '42 0 7', '57 7 4'],
 ['water-to-light map:', '88 18 7', '18 25 70'],
 ['light-to-temperature map:', '45 77 23', '81 45 19', '68 64 13'],
 ['temperature-to-humidity map:', '0 69 1', '1 0 69'],
 ['humidity-to-location map:', '60 56 37', '56 93 4']]

In [ ]:
m = ys[1]
m

['seed-to-soil map:', '50 98 2', '52 50 48']

In [ ]:
src, dst = m[0].split()[0].split('-to-')
src, dst

('seed', 'soil')

In [ ]:
def parse_src_dst(s): return s.split()[0].split('-to-')
parse_src_dst(m[0])

['seed', 'soil']

In [ ]:
L(m[1].split()).map(int)

[50, 98, 2]

In [ ]:
def to_num_list(s): return L(s.split()).map(int)
to_num_list(m[1])

[50, 98, 2]

In [ ]:
def parse_map(lines):
    src, dst = parse_src_dst(lines[0])
    num_lists = L(lines[1:]).map(to_num_list)
    return src, dst, num_lists
src, dst, ns = parse_map(m)
src, dst, ns

('seed', 'soil', [[50, 98, 2], [52, 50, 48]])

In [ ]:
rd = RangeDict()
for n in ns: rd.add_range(*n)
L(79, 14, 55, 13).map(rd.__getitem__)

[81, 14, 57, 13]

In [ ]:
from types import SimpleNamespace as ns

src, dst, nums = parse_map(m)

def create_map(src, dst, nums):
    rd = RangeDict()
    for n in nums: rd.add_range(*n)
    return ns(src=src, dst=dst, rd=rd)
mp = create_map(src, dst, nums)
mp

namespace(src='seed', dst='soil', rd=<__main__.RangeDict at 0x75f509c67410>)

In [ ]:
def parse_seeds(s): return to_num_list(s.removeprefix("seeds: "))
parse_seeds("seeds: 79 14 55 13")

[79, 14, 55, 13]

In [ ]:
maps = L(ys[1:]).map(parse_map).starmap(create_map)
maps

[namespace(src='seed', dst='soil', rd=<__main__.RangeDict object at 0x75f509677740>), namespace(src='soil', dst='fertilizer', rd=<__main__.RangeDict object at 0x75f509675c40>), namespace(src='fertilizer', dst='water', rd=<__main__.RangeDict object at 0x75f509674b60>), namespace(src='water', dst='light', rd=<__main__.RangeDict object at 0x75f509677350>), namespace(src='light', dst='temperature', rd=<__main__.RangeDict object at 0x75f5096770b0>), namespace(src='temperature', dst='humidity', rd=<__main__.RangeDict object at 0x75f5096772c0>), namespace(src='humidity', dst='location', rd=<__main__.RangeDict object at 0x75f509676360>)]

In [ ]:
current = "seed"
current_num = 79
while current != "location":
    m = maps.filter(lambda x: x.src==current)[0]
    current = m.dst
    current_num = m.rd[current_num]
current, current_num

('location', 82)

In [ ]:
def find_location(seed_num, maps):
    current = "seed"
    current_num = seed_num
    while current != "location":
        m = maps.filter(lambda x: x.src==current)[0]
        current = m.dst
        current_num = m.rd[current_num]
    return current_num

for i in [79, 14, 55, 13]:
    print(find_location(i, maps))

82
43
86
35


In [ ]:
def solve(lines):
    seeds = parse_seeds(lines[0])
    ys = split_on(lines[1:], '')
    maps = L(ys).map(parse_map).starmap(create_map)
    locs = L(seeds).map(find_location, maps=maps)
    return min(locs)
solve(samp.splitlines())

35

In [ ]:
solve(inp.splitlines())

331445006

# Part 2

In [ ]:
from fastcore.utils import chunked

chunks = L(chunked(L(parse_seeds("seeds: 79 14 55 13")), chunk_sz=2))
chunks

[[79, 14], [55, 13]]

In [ ]:
chunks[0][0], chunks[0][1]

(79, 14)

In [ ]:
list(range(79, 79+14))

[79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92]

In [ ]:
c = chunks.map(lambda t: list(range(t[0], t[0] + t[1]))).concat()
c

[79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67]

In [ ]:
def solve_part2(lines):
    chunks = L(chunked(L(parse_seeds(lines[0])), chunk_sz=2))
    seeds = chunks.map(lambda t: list(range(t[0], t[0] + t[1]))).concat()
    
    ys = split_on(lines[1:], '')
    maps = L(ys).map(parse_map).starmap(create_map)
    locs = L(seeds).map(find_location, maps=maps)
    return min(locs)
solve_part2(samp.splitlines())

46

In [ ]:
# solve_part2(inp.splitlines())

In [ ]:
lines = inp.splitlines() 
chunks = L(chunked(L(parse_seeds(lines[0])), chunk_sz=2))
ranges = chunks.map(lambda t: range(t[0], t[0] + t[1]))
ranges

# seeds = chunks.map(lambda t: list(range(t[0], t[0] + t[1]))).concat()
# seeds


[range(487758422, 1012095270), range(2531594804, 2558702571), range(1343486056, 1467813607), range(1117929819, 1211026889), range(3305050822, 3747371247), range(2324984130, 2412588554), range(4216329536, 4261368470), range(1482842780, 1707453678), range(115202033, 486534091), range(2845474954, 3038054813)]

How can I merge ranges?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

To merge overlapping or adjacent ranges, you'll want to:

1. Sort the ranges by their start values
2. Iterate through and merge ranges that overlap or touch
3. Keep track of the current merged range

Here's a small function to do this:

```python
def merge_ranges(ranges):
    if not ranges: return []
    sorted_ranges = sorted(ranges, key=lambda r: r.start)
    merged = [sorted_ranges[0]]
    
    for current in sorted_ranges[1:]:
        last = merged[-1]
        if current.start <= last.stop:
            merged[-1] = range(last.start, max(last.stop, current.stop))
        else:
            merged.append(current)
    return merged
```

However, looking at your code - are you trying to merge the seed ranges, or are you running into a performance issue with the large ranges? Those ranges are huge (hundreds of millions of values), so you probably don't want to materialize them as lists. 

What's your goal with merging the ranges?

I want to remove duplicated values

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

For removing duplicate values, you have a few options depending on what you're working with:

1. **If you have actual lists of numbers** - Convert to a set: `set(your_list)`
2. **If you have range objects** - Ranges don't contain duplicates by definition, but overlapping ranges might cover the same numbers

Looking at your code, I see you have range objects that might overlap. Are you trying to:
- Remove overlapping coverage between ranges (so no number appears in multiple ranges)?
- Or something else?

What exactly are you trying to achieve with the seed ranges?

I'm trying to remove overlapping coverage between ranges

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

To remove overlapping coverage, you'll want to merge the ranges so each number is only covered once. The function I showed earlier does this, but it needs a small fix for range objects since `range.stop` is exclusive.

Here's the corrected version:

```python
def merge_ranges(ranges):
    if not ranges: return []
    sorted_ranges = sorted(ranges, key=lambda r: r.start)
    merged = [sorted_ranges[0]]
    
    for current in sorted_ranges[1:]:
        last = merged[-1]
        if current.start <= last.stop:
            merged[-1] = range(last.start, max(last.stop, current.stop))
        else:
            merged.append(current)
    return merged
```

Try applying this to your chunks ranges. Does this give you non-overlapping ranges?

In [ ]:
def merge_ranges(ranges):
    if not ranges: return []
    sorted_ranges = sorted(ranges, key=lambda r: r.start)
    merged = [sorted_ranges[0]]
    
    for current in sorted_ranges[1:]:
        last = merged[-1]
        if current.start <= last.stop:
            merged[-1] = range(last.start, max(last.stop, current.stop))
        else:
            merged.append(current)
    return merged
merge_ranges(ranges)

[range(115202033, 486534091),
 range(487758422, 1012095270),
 range(1117929819, 1211026889),
 range(1343486056, 1467813607),
 range(1482842780, 1707453678),
 range(2324984130, 2412588554),
 range(2531594804, 2558702571),
 range(2845474954, 3038054813),
 range(3305050822, 3747371247),
 range(4216329536, 4261368470)]

How can I create a generator which will return a number from the list of ranges?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You can chain the ranges together using `itertools.chain`. Here's how:

```python
from itertools import chain
gen = chain.from_iterable(ranges)
```

This creates a generator that yields numbers from the first range, then the second, and so on, without materializing all the values at once.

Would you like to use this to iterate through all the seed numbers for part 2?

In [ ]:
from itertools import chain
gen = chain.from_iterable([range(0, 3), range(10, 15)])
min(gen)

0

In [ ]:
def solve_part2(lines):
    chunks = L(chunked(L(parse_seeds(lines[0])), chunk_sz=2))
    ranges = chunks.map(lambda t: range(t[0], t[0] + t[1]))
    min_ranges = merge_ranges(ranges)
    seeds = chain.from_iterable(min_ranges)
    
    ys = split_on(lines[1:], '')
    maps = L(ys).map(parse_map).starmap(create_map)

    locs = map(lambda s: find_location(s, maps=maps), seeds)
    return min(locs)

solve_part2(samp.splitlines())

46

In [ ]:
# solve_part2(inp.splitlines())

How can I optimize the solve_part2 function? It seems to have a problem with large inputs.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

The issue is that you're iterating through billions of individual seed numbers, which is too slow. Instead of mapping individual numbers through the ranges, you need to map entire ranges through the transformations.

The key insight: instead of processing each seed number individually, track ranges as they transform through each map. When a range goes through a `RangeDict`, it might:
- Stay entirely within one mapping rule
- Span across multiple mapping rules
- Have parts that map and parts that don't

You'll need to modify your approach to:
1. Keep seed ranges as range objects (not individual numbers)
2. Split ranges when they cross mapping boundaries
3. Transform the ranges through each map

This is more complex than the individual number approach. Would you like a hint on how to implement range splitting, or would you prefer to think about the algorithm first?

In [ ]:
class RangeDict:
    def __init__(self):
        self.ranges = []
    
    def add_range(self, dest_start, source_start, length):
        self.ranges.append((dest_start, source_start, length))
    
    def __getitem__(self, key):
        for dest_start, source_start, length in self.ranges:
            if source_start <= key < source_start + length:
                offset = key - source_start
                return dest_start + offset
        return key  # If no mapping found, return the key itself
    
    def get_range(source_start, length):
        pass